## Scarping insolvency data
Source: https://www.insolvenzbekanntmachungen.de/cgi-bin/bl_suche.pl

1. 'Uneingeschränkte Suche': search for all insolvency court but only for 2 weeks after publication date
2. 'Detail-Suche': 
    * input: insolvency court + company name OR residence of the debtor



In [1]:
# install required packages:
#!pip install requests
#!pip install bs4
#!pip install pandas

In [2]:
# import required packages:
import json
import helpers

# 1. Get regions and insolvency court from
www.insolvenzbekanntmachungen.de:

In [3]:
insolvency_court = helpers.scrape_insolvency_court()

In [4]:
insolvency_court

{'Baden-Württemberg': ['Aalen',
  'Baden-Baden',
  'Crailsheim',
  'Esslingen',
  'Freiburg',
  'Göppingen',
  'Hechingen',
  'Heidelberg',
  'Heilbronn',
  'Karlsruhe',
  'Konstanz',
  'Ludwigsburg',
  'Lörrach',
  'Mannheim',
  'Mosbach',
  'Offenburg',
  'Pforzheim',
  'Ravensburg',
  'Rottweil',
  'Stuttgart',
  'Tübingen',
  'Ulm',
  'Villingen-Schwenningen',
  'Waldshut-Tiengen'],
 'Bayern': ['Amberg',
  'Ansbach',
  'Aschaffenburg',
  'Augsburg',
  'Bamberg',
  'Bayreuth',
  'Coburg',
  'Deggendorf',
  'Fürth',
  'Hof',
  'Ingolstadt',
  'Kempten',
  'Landshut',
  'Memmingen',
  'Mühldorf',
  'München',
  'Neu-Ulm',
  'Nördlingen',
  'Nürnberg',
  'Passau',
  'Regensburg',
  'Rosenheim',
  'Schweinfurt',
  'Straubing',
  'Traunstein',
  'Weiden',
  'Weilheim',
  'Wolfratshausen',
  'Würzburg'],
 'Berlin': ['Charlottenburg',
  'Köpenick',
  'Lichtenberg',
  'Mitte',
  'Neukölln',
  'Pankow/Weißensee',
  'Schöneberg',
  'Spandau',
  'Tempelhof-Kreuzberg',
  'Tiergarten',
  'Weddin

# 2. Example:

## 2.1 unlimited_search():

In [5]:
# helpers.unlimited_search(region='--+Alle+Bundesl%E4nder+--', court='--+Alle+Insolvenzgerichte+--&', 
#                          matchesperpage=100, verbose=False)

## 2.1 detail_search():

In [6]:
helpers.detail_search(name='Senvion GmbH', region='Hamburg', court='Hamburg', verbose=True)

--- 0.3107028007507324 seconds ---
Number of results: 5


['2020-06-03Senvion GmbH, Hamburg, 67g IN 113/19, Registergericht Hamburg, HRB 137187',
 '2020-04-15Senvion GmbH, Hamburg, 67g IN 113/19, Registergericht Hamburg, HRB 137187',
 '2019-08-22Senvion GmbH, Hamburg, 67g IN 113/19, Registergericht Hamburg, HRB 137187',
 '2019-07-02Senvion GmbH, Hamburg, 67g IN 113/19, Registergericht Hamburg, HRB 137187',
 '2019-07-02Senvion GmbH, Hamburg, 67g IN 113/19, Registergericht Hamburg, HRB 137187']

## 3. Get courts:
www.gerichtsverzeichnis.de

## 

In [7]:
# company attributes: [Name, Mitarbeiter, Umsatz in Mio, PLZ, Stadt]

# company_attributes_big: 13.06.2020
file_name = 'company_attributes_big'

with open('data/' + file_name + '.json','r') as file:
    company_attributes = json.load(file)

In [8]:
len(company_attributes)

4167

In [9]:
insolvenz_data_dict, errors = helpers.scrape_court_and_insolvency(company_attributes, insolvency_court)

In [10]:
len(insolvenz_data_dict)

137

In [11]:
len(errors)

82

In [12]:
insolvenz_data_dict.keys()

dict_keys(['Baumann', 'Spie', 'Schickor Personaldienstleistungs GmbH', 'TelDaFax Network GmbH', 'INOVA Competence GmbH', 'Baumarkt Max Bahr Praktiker Einkaufs GmbH', 'we plan GmbH', 'Thomas Binder', 'peesel', 'WKD Sicherheit GmbH', 'SSV Sicherheitstechnik GmbH', 'DynaTrav Dynamic Travel Solutions GmbH', 'P P Personal-Profi GmbH', 'Speedwork Logistics GmbH', 'Riedl', 'JUMA Job GmbH', 'i-tegra xperts GmbH', 'Contact Personal Service GmbH', 'besser Personalmanagement GmbH', 'schirmherrschaft gmbh', 'Bunjes', 'Vision Event GmbH', 'GETS-Logistik GmbH', 'Riede Stefan', "Theo's Personal Vermietungs-Service GmbH", 'deutscher spendenhilfsdienst - DSH GmbH', 'Spa Group Europe GmbH', 'Kristall Personalservice GmbH', 'kipping', 'spie', 'Raabe', 'NCL', 'persona', 'Lemmerz Personalservice GmbH', 'Personal Optimal GmbH', 'Wegmeyer', 'frinch AG', 'Valet One GmbH', 'Trudwig GmbH & Co. KG', 'Barthelmess GmbH', 'BELL EMPLOYMENT PERSONALDIENSTLEISTUNGEN GmbH', 'jr world business gmbh', 'Infinus AG Ihr Kom

In [14]:
with open('data/' + file_name + '_insolvenz_data.json', 'w') as f:
    json.dump(insolvenz_data_dict, f)

In [15]:
with open('data/' + file_name + '_errors_.json', 'w') as f:
    json.dump(errors, f)

In [16]:
with open('data/' + file_name + '_insolvenz_data.json','r') as file:
    test = json.load(file)

In [17]:
test

{'Baumann': ['2020-05-18Baumann Sven Gerhard, 93133 Burglengenfeld, 483 IK 411/14',
  '2020-05-06Baumann Sven Gerhard, 93133 Burglengenfeld, 483 IK 411/14',
  '2020-04-02Baumann Sven Gerhard, 93133 Burglengenfeld, 483 IK 411/14',
  '2019-10-25Baumann Ludwig, 92421 Schwandorf, IN 296/16',
  '2019-10-22Baumann Ludwig, 92421 Schwandorf, IN 296/16',
  '2019-10-21Baumann Ludwig, 92421 Schwandorf, IN 296/16',
  '2019-09-17Baumann Ludwig, 92421 Schwandorf, IN 296/16',
  '2019-05-07Baumann Barbara Margarete, 92421 Schwandorf, IK 289/18',
  '2019-01-21Ludwig Baumann, Schwandorf, 263 IN 65/15',
  '2019-01-11Baumann Ludwig, Schwandorf, 263 IN 65/15',
  '2019-01-11Baumann Ludwig, Schwandorf, 263 IN 65/15',
  '2018-11-23Baumann Ludwig, Schwandorf, 263 IN 65/15',
  '2018-09-18Baumann Barbara Margarete, 92421 Schwandorf, IK 289/18',
  '2017-08-10Baumann Ludwig, 92421 Schwandorf, IN 296/16',
  '2017-08-04Baumann Ludwig, 92421 Schwandorf, 263 IN 65/15',
  '2017-01-25Baumann, Tanja, Freudenberg,  IK 125

# End

## additional approach -> NOT WORKING!
1. detailed search for:


* GbR
* AG
* GmbH
* UG
* Ltd.
* OHG

-> not included 'Einzelunternehmen'


In [52]:
company_name_shortcuts = ['AG', 'GbR', 'KG', 'GmbH', 'UG', 'Ltd.', 'OHG']
all_insolvency_shortcuts = {}
for region in insolvency_court:
    for court in insolvency_court[region]:
        for name in company_name_shortcuts:
            #print(region)
            #print(city)                    
            print(name + ': ' + region + ': ' +court)
            data = unlimited_search(name=name, region=region, court=court, search_type='Suchfunktion=detail&Absenden=Suche+starten&')
            if len(data) > 0:
                all_insolvency_shortcuts[name] = data
                
    break

AG: Baden-Württemberg: Aalen


IndexError: list index out of range

In [53]:
name = 'AG'
region = 'Baden-Württemberg'
court = 'Aalen'

search_type='Suchfunktion=detail&Absenden=Suche+starten&'
matchesperpage = '10'

URL = 'https://www.insolvenzbekanntmachungen.de/cgi-bin/bl_suche.pl'

# 1. find max number of pages
with requests.Session() as s:
    s.headers = {"User-Agent": "Mozilla/5.0"}
    s.headers.update({'Content-Type': 'application/x-www-form-urlencoded'})
    payload = search_type + \
              'Bundesland=' + region + \
              '&Gericht=' + court + \
              'Datum1=&Datum2=' \
              '&Name=' + name + \
              '&Sitz=' + \
              '&Abteilungsnr=' + \
              '&Registerzeichen=--&Lfdnr=' + \
              '&Jahreszahl=--' + \
              '&Registerart=--+keine+Angabe+--' + \
              '&select_registergericht=' + \
              '&Registergericht=--+keine+Angabe+--' + \
              '&Registernummer=' + \
              '&Gegenstand=--+Alle+Bekanntmachungen+innerhalb+des+Verfahrens+--' + \
              '&matchesperpage=' + str(matchesperpage) + \
              '&page=1' + \
              '&sortedby=Datum'
    res = s.post(URL, data=payload)
    soup = BeautifulSoup(res.text, "lxml")
    t = soup.select('center a')[-1].attrs['href']

IndexError: list index out of range

# -- old - not used anymore --

# 4. Try to structure data in data frame:

In [27]:
real_person = pd.DataFrame({"Date": [], 
                            "Last Name": [],
                            "First Name": [],
                            "Address": [],
                            "Location": [],
                            "Reference Number": [],
                            "Raw Information": []}) 

In [28]:
companies = pd.DataFrame({"Date": [], 
                          "Company Name": [],
                          "Residence ": [],
                          "Reference Number": [],
                          "Raw Information": []}) 

In [30]:
for idx, entry in enumerate(insolvency_data):
    date = entry[0:10]
    entry_list = entry[10:].split(", ")

    # real person without address
    if len(entry_list) == 4:
        real_person.loc[idx] = [date, entry_list[0], entry_list[1], entry_list[2], '-', entry_list[3], entry]
    # real person with full address
    elif len(entry_list) == 5:
        real_person.loc[idx] = [date, entry_list[0], entry_list[1], entry_list[2], entry_list[3], entry_list[4], entry]
    # company name
    elif len(entry_list) == 3:
        companies.loc[idx] = [date, entry_list[0], entry_list[1], entry_list[2], entry]
    else:
        real_person.loc[idx] = [date, entry_list[0], entry_list[1], '?', '?', '?', entry]
        #print(entry_list)

In [31]:
companies.shape

(4908, 5)

In [32]:
real_person.shape

(38792, 7)

In [33]:
companies

,Date,Company Name,Residence,Reference Number,Raw Information
20,2020-05-30,Alexa Balko,Cottbus,64 IK 106/14,"2020-05-30Alexa Balko, Cottbus, 64 IK 106/14"
21,2020-05-30,Alexa Balko,Cottbus,64 IK 106/14,"2020-05-30Alexa Balko, Cottbus, 64 IK 106/14"
23,2020-05-30,Andreas Lux,Schönefeld,64 IK 286/14,"2020-05-30Andreas Lux, Schönefeld, 64 IK 286/14"
24,2020-05-30,Andreas Lux,Schönefeld,64 IK 286/14,"2020-05-30Andreas Lux, Schönefeld, 64 IK 286/14"
36,2020-05-30,Björn Thieme,Bad Liebenwerda,64 IK 692/18,"2020-05-30Björn Thieme, Bad Liebenwerda, 64 IK..."
...,...,...,...,...,...
43683,2020-05-18,Yusuf Sahin,Geislingen,4 IK 279/19,"2020-05-18Yusuf Sahin, Geislingen, 4 IK 279/19"
43684,2020-05-18,Yusuf Sahin,Geislingen,4 IK 279/19,"2020-05-18Yusuf Sahin, Geislingen, 4 IK 279/19"
43696,2020-05-18,Zecevic Jana,München,1513 IK 693/19,"2020-05-18Zecevic Jana, München, 1513 IK 693/19"
43698,2020-05-18,Zenebe Tebeje,Heidelberg,82 IK 211/19,"2020-05-18Zenebe Tebeje, Heidelberg, 82 IK 211/19"


In [34]:
real_person

,Date,Last Name,First Name,Address,Location,Reference Number,Raw Information
0,2020-05-31,Ali Panah,Mohamad Reza,Hannover,-,903 IK 1012/19 - 3 -,"2020-05-31Ali Panah, Mohamad Reza, Hannover, 9..."
1,2020-05-31,Ali Panah,Mohamad Reza,Hannover,-,903 IK 1012/19 - 3 -,"2020-05-31Ali Panah, Mohamad Reza, Hannover, 9..."
2,2020-05-31,Ali Panah,Mohamad Reza,Hannover,-,903 IK 1012/19 - 3 -,"2020-05-31Ali Panah, Mohamad Reza, Hannover, 9..."
3,2020-05-31,Burdorf,Dominika,Lilienthal,-,11 IK 76/20,"2020-05-31Burdorf, Dominika, Lilienthal, 11 IK..."
4,2020-05-31,Burdorf,Dominika,Lilienthal,-,11 IK 76/20,"2020-05-31Burdorf, Dominika, Lilienthal, 11 IK..."
...,...,...,...,...,...,...,...
43692,2020-05-18,Zahapschi,Vasile,Neulußheim,-,2 IK 1122/19,"2020-05-18Zahapschi, Vasile, Neulußheim, 2 IK ..."
43693,2020-05-18,Zahn,Irene,Kiel,-,24 IK 191/14,"2020-05-18Zahn, Irene, Kiel, 24 IK 191/14"
43694,2020-05-18,Zapf geb. Suhling,Marita,Ludwigshafen,-,3 a IK 136/14 Lu,"2020-05-18Zapf geb. Suhling, Marita, Ludwigsha..."
43695,2020-05-18,Zbierski,Thomas,23968 Wismar,-,580 IK 349/19,"2020-05-18Zbierski, Thomas, 23968 Wismar, 580 ..."


# 5. Save data

In [33]:
# save as .pkl file
real_person.to_pickle('Insolvency/real_person_'+ region +'.pkl') 
companies.to_pickle('Insolvency/companies_'+ region +'.pkl') 

In [34]:
# save as .json file
with open('Insolvency/insolvency_'+ region +'.json','w') as f:
   json.dump(list(insolvency),f)